<a href="https://colab.research.google.com/github/helenason/CNN-shallowNet/blob/main/%EC%86%90%EC%B1%84%EC%98%81_shallowNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
filname = '/content/gdrive/MyDrive/Colab Notebooks/data/fer2013.csv'

Mounted at /content/gdrive


In [3]:
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names = ['emotion','pixels', 'usage']
df = pd.read_csv(filname,names=names, na_filter=False)
im = df['pixels']
df.head(10)
df.shape

(35888, 3)

In [4]:
def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) / 255.0 for p in row[1].split()])

    X, Y = np.array(X), np.array(Y)
    return X, Y

In [5]:
X, Y = getData(filname)
print(X.shape)
print(Y.shape)
num_class = len(set(Y))
print(num_class)

(35887, 2304)
(35887,)
7


In [6]:
N, D = X.shape
X = X.reshape(N, 48, 48, 1)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)

In [8]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
# from keras.layers.normalization import BatchNormalization
from keras.layers import BatchNormalization
# from keras import backend as K

In [20]:
# INPUT => CONV => MaxPooling => RELU => FC 

def shallow_model():
    model = Sequential()
    input_shape = (48, 48, 1)
    model.add(Conv2D(64, (5, 5), padding='same', activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(Dropout(.2, input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(7))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return model

model = shallow_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        1664      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 64)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 36864)             0         
                                                                 
 dense_1 (Dense)             (None, 7)                 258055    
                                                                 
 activation_1 (Activation)   (None, 7)                 0         
                                                      

In [26]:
path_model='model_filter.h5' # save model at this location after each epoch
keras.backend.clear_session() # destroys the current graph and builds a new one
model = shallow_model() # create the model
keras.backend.set_value(model.optimizer.lr, 1e-3) # set the learning rate

h = model.fit(x=X_train,     
            y=y_train, 
            batch_size=64, 
            epochs=20, 
            verbose=1, 
            validation_data=(X_test,y_test),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
)

Epoch 1/20
505/505 [==============================] - 5s 8ms/step - loss: 1.6518 - accuracy: 0.3501 - val_loss: 1.5302 - val_accuracy: 0.4174
Epoch 2/20
505/505 [==============================] - 4s 7ms/step - loss: 1.4825 - accuracy: 0.4364 - val_loss: 1.4728 - val_accuracy: 0.4466
Epoch 3/20
505/505 [==============================] - 3s 7ms/step - loss: 1.3936 - accuracy: 0.4730 - val_loss: 1.4279 - val_accuracy: 0.4606
Epoch 4/20
505/505 [==============================] - 3s 6ms/step - loss: 1.3254 - accuracy: 0.5007 - val_loss: 1.4335 - val_accuracy: 0.4692
Epoch 5/20
505/505 [==============================] - 4s 7ms/step - loss: 1.2717 - accuracy: 0.5272 - val_loss: 1.3928 - val_accuracy: 0.4915
Epoch 6/20
505/505 [==============================] - 4s 7ms/step - loss: 1.2161 - accuracy: 0.5483 - val_loss: 1.4096 - val_accuracy: 0.4829
Epoch 7/20
505/505 [==============================] - 3s 6ms/step - loss: 1.1721 - accuracy: 0.5664 - val_loss: 1.4015 - val_accuracy: 0.4929
Epoch 

In [11]:
# objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
# y_pos = np.arange(len(objects))
# print(y_pos)

In [12]:
def emotion_analysis(emotions):
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    y_pos = np.arange(len(objects))

    plt.bar(y_pos, emotions, align='center', alpha=0.9)
    plt.tick_params(axis='x', which='both', pad=10,width=4,length=10)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
    plt.show()

In [13]:
y_pred=model.predict(X_test)
#print(y_pred)
y_test.shape

113/113 [==============================] - 0s 2ms/step


(3589, 7)

In [14]:
# 민지님이 올려주신 자료로 테스트

import keras.utils as image

def showResult(addr, label) :

    img = image.load_img(addr, color_mode = "grayscale", target_size=(48, 48))
    show_img=image.load_img(addr, target_size=(200, 200))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)

    x /= 255

    custom = model.predict(x)
    # # print(custom[0])
    # emotion_analysis(custom[0])

    x = np.array(x, 'float32')
    x = x.reshape([48, 48]);

    # plt.gray()
    # plt.imshow(show_img)
    # plt.show()

    max_idx = 0
    m = 0.000000000000000000001
    a = custom[0]
    for i in range(0, len(a)): # max 찾기
        if a[i] > m:
            m = a[i]
            max_idx = i
            
    # print('Expression Prediction:',objects[idx])

    if max_idx == label :
        return 1
    else :
        return 0


import os

objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

res = 0
num_images = 0

for i, obj in enumerate(objects) :

    label = i

    address = f'/content/gdrive/MyDrive/Colab Notebooks/data/archive/test/test/{obj}/'

    images = os.listdir(address)

    num_images += len(images)

    for img in images :
        res += showResult(address + img, label)

print(num_images)
print(res / num_images * 100)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 24ms/step
7155
82.68343815513627
